# Running ASFMAUS

In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays
using Distributions

In [2]:
directory_path = "/home/callum/ASF/"

"/home/callum/ASF/"

In [16]:
include("$(directory_path)Modules/ASF_input.jl");
include("$(directory_path)Modules/ASF_model.jl");
include("$(directory_path)Modules/ASF_analysis.jl");

In [4]:
input_1 = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

In [5]:
nt = input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 10#number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1

dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [6]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
                ASF_Model.reparam!(input_1)
                remake(prob,p=input_1.Parameters)
end 

prob_func (generic function with 1 method)

## Running the Model

### Frequency run

In [ ]:
#single run with frequency inter group interections
rj = RegularJump(ASF_Model.frequency_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output_f = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 20,dt=.1);

In [ ]:
ASF_Analysis.analyse_out(output_f, input_1.Parameters.Populations)

### Density run

In [ ]:

rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)
#ensemble run with density inter group interections

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)
#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 100,EnsembleThreads(),dt=1);

In [ ]:
ASF_Analysis.analyse_out(output, input_1.Parameters.Populations)

## Running Multiple Networks

In [ ]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    dir = "$(path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    for i = 1:n_networks

        inputs = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs, inputs.Parameters.Populations) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j)",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
    end
    
end

In [ ]:
sensitivity(20,20,100,directory_path, "Frequency") # running 

## Analysing Output

In [7]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Density/");
frequency = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Frequency/");


In [17]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

Exposed!
Mean- 91.38 & 91.25 - 91.51

Alive!
Mean- 30.48 & 30.37 - 30.58

Time!
Mean- 243.42 & 242.76 - 244.09

Filtered

Exposed!
Mean- 93.26 & 93.23 - 93.29

Alive!
Mean- 29.06 & 29.01 - 29.11

Time!
Mean- 247.31 & 246.68 - 247.93


In [19]:
frequency_r = ASF_Analysis.three_statistics(frequency,20, p_out = true);

Exposed!
Mean- 98.03 & 97.96 - 98.09

Alive!
Mean- 23.61 & 23.54 - 23.67

Time!
Mean- 220.31 & 219.89 - 220.73

Filtered

Exposed!
Mean- 98.48 & 98.47 - 98.49

Alive!
Mean- 23.26 & 23.21 - 23.3

Time!
Mean- 221.15 & 220.74 - 221.55
